In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_black
%load_ext nb_mypy

In [ ]:
import nbimporter

# Stockfish Spieler

Dieses Notebook implementiert die Verbindung zur Stockfish-Engine als Spieler, welcher dann innerhalb eines definierten Zeitlimits nach einem möglichst guten Zug sucht.

In [ ]:
import chess
import chess.engine  # type: ignore
from AIBaseClass import ChessAI


class StockfishPlayer(ChessAI):
    def __init__(self, elo=1500, time_limit=0.1, **kwargs) -> None:
        super().__init__(**kwargs)
        self.time_limit = time_limit
        self.elo = elo
        self.initialize_engine()

    def initialize_engine(self) -> None:
        """Intializes the engine communication."""
        self.engine = chess.engine.SimpleEngine.popen_uci(
            "../lib/stockfish/stockfish.exe"
        )
        self.engine.configure({"UCI_LimitStrength": True, "UCI_Elo": self.elo})

    def reset(self: ChessAI) -> None:
        """Close engine communication"""
        self.engine.quit()
        self.engine = None
        super().reset()

    def get_next_middle_game_move(self: ChessAI, board: chess.Board) -> chess.Move:
        """Communicate with the stockfish engine over uci."""
        if self.engine is None:
            self.initialize_engine()
        # Limit stockfish thinking time
        result = self.engine.play(
            board, chess.engine.Limit(self.time_limit)
        )  # type:ignore
        return result.move

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
# Create player and board
unit_test_player = StockfishPlayer(player_name="Stockfish", elo=2000, time_limit=0.2)
board = chess.Board()

In [ ]:
# Test next move function
move = unit_test_player.get_next_middle_game_move(board)
assert move in board.legal_moves, "Illegal move received!"
move

In [ ]:
# Test reset function
unit_test_player.reset()
assert unit_test_player.engine is None, "Reset was not successful!"